In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from matplotlib.animation import FuncAnimation
from sklearn.model_selection import train_test_split

In [13]:
##Wavenumbers
k = 12 * np.pi
m = 6 * np.pi
m_U = 14 * np.pi #vertical wavenumber
##

r_m = 0.1
N0_squared = 100


k_e = np.sqrt(k**2 + m**2)
k_plus = np.sqrt(k**2 +(m+m_U)**2)

W_e = np.array([[-1, k / (k_e**2)],
                [-k * N0_squared, -1]])
W_plus = np.array([[-1, -k / (k_plus**2)],
                [k * N0_squared, -1]])

W = np.block([[W_e, np.zeros((2, 2))],
             [np.zeros((2, 2)), W_plus]])

L_e = np.array([[-(k*(k_plus**2 - m_U**2)) / (2 * (k_e**2)), 0],
                [0, k / 2]])

L_plus = np.array([[-(k*(m_U**2 - k_e**2)) / (2 * (k_plus**2)), 0],
                [0, -k / 2]])

L = np.block([[np.zeros((2, 2)), L_e],
             [L_plus, np.zeros((2, 2))]])

In [14]:
initial_state = np.array([[0, 0, 0, 0, 0.001]]).T  #The initial state for the y_em output where the concatenated array looks like [psi_e, b_e, psi_plus, b_plus, U]

t_span = (0, 200)  #the range of time units
dt = 0.001  #the timestep (dt=0.001 is a pretty decent value to stay at)

epsilon = 0.01  #noise strength (0.01 has been well used)

#The function below uses the Euler Maruyama method which is used to solve Stochastic Differential Equations (SDEs)
def euler_maruyama(y0, t_span, dt, epsilon):
    t0, tf = t_span
    t = np.linspace(t0, tf, int((tf - t0) / dt) + 1)
    s = np.zeros((4, len(t)))
    U = np.zeros(len(t))
    R = np.zeros(len(t))

    s[:, 0] = y0[:4].ravel()
    U[0] = y0[4]
    R[0] = 0

    for i in range(len(t) - 1):
        noise_forcing = np.sqrt(dt) * (2 * np.sqrt(2) / k_e) * np.random.normal(0, 1)
        noise_forcing_vector = np.array([noise_forcing, 0, 0, 0])
        psi_e = s[0, i]
        psi_p = s[2, i]
        s[:, i + 1] = s[:, i] + dt * (W @ s[:, i] + U[i] * (L @ s[:, i])) + np.sqrt(epsilon) * noise_forcing_vector
        U[i + 1] = U[i] + dt * ((0.25 * k * (k_plus**2 - k_e**2) * psi_e * psi_p) - (r_m * U[i]))

    y = np.vstack((s, U))
    return t, y

In [7]:
num_data_points = 5
data_points = []

for i in range(num_data_points):
    # Set a different random seed for each run
    np.random.seed(i)
    
    # Run the euler_maruyama function with the same parameters as before
    t_em, y_em = euler_maruyama(initial_state, t_span, dt, epsilon)

    # Combine the time, R_value_em, and y_em[4] into a single numpy array
    data = np.column_stack((t_em, y_em[4]))

    # Append the data to the data_points list
    data_points.append(data)

# Concatenate all the data points into a single numpy array
data_points = np.vstack(data_points)

# Create a DataFrame from the data
df = pd.DataFrame(data_points, columns=['Time', 'U'])

# Shuffle the data
#df = df.sample(frac=1).reset_index(drop=True)

# Split the data into training and test sets (80% training, 20% test)
train_frac = 0.8
train_size = int(len(df) * train_frac)
train_df = df[:train_size]
test_df = df[train_size:]

# Save the DataFrames as .csv files
train_df.to_csv('training_data_5.csv', index=False)
test_df.to_csv('test_data_5.csv', index=False)

In [4]:
def generate_data(num_samples, y0, t_span, dt, epsilon):
    t_list = []
    y_list = []

    for _ in range(num_samples):
        t_em, y_em = euler_maruyama(y0, t_span, dt, epsilon)
        t_list.append(t_em)
        y_list.append(y_em[4])  # Extracting only y_em[4] as specified

    return np.array(t_list), np.array(y_list)

In [10]:
def generate_data(num_samples, y0, t_span, dt, epsilon):
    t_list = []
    y_list = []

    for _ in range(num_samples):
        t_em, y_em = euler_maruyama(y0, t_span, dt, epsilon)
        t_list.append(t_em)
        y_list.append(y_em[4])  # Extracting only y_em[4] as specified

    # Convert the lists to float32 numpy arrays and reshape the data
    t_array = np.array(t_list, dtype=np.float32).reshape(-1, 1)
    y_array = np.array(y_list, dtype=np.float32).reshape(-1, 1)

    return t_array, y_array

In [11]:
# Generate 1000 data samples
num_samples = 10
t_data, y_data = generate_data(num_samples, initial_state, t_span, dt, epsilon)

# Combine the data into a DataFrame
data = {'t': t_data.flatten(), 'y_em': y_data.flatten()}
df = pd.DataFrame(data)

# Save the DataFrame to a .csv file
df.to_csv('./data/generated_data.csv', index=False)

# Split the data into 40% training and 60% testing
train_ratio = 0.4
test_ratio = 0.6
train_df, test_df = train_test_split(df, train_size=train_ratio, test_size=test_ratio, random_state=42)

# Save the training and testing data to .csv files
train_df.to_csv('./data/train_data_new.csv', index=False)
test_df.to_csv('./data/test_data_new.csv', index=False)

In [16]:
t, y = euler_maruyama(initial_state, t_span, dt, epsilon)

# Save the results to a DataFrame and export it to a CSV file
df = pd.DataFrame(np.hstack([t[:, np.newaxis], y.T]), columns=['t', 'psi_e', 'b_e', 'psi_plus', 'b_plus', 'U'])
df.to_csv('./data/train_data.csv', index=False)